# 4. 하둡을 활용한 데이터 입수
이장에서 다룰 내용
- 새로운 데이터 처리 패러다임으로 부상한 데이터 레이크의 개념
- CSV 데이터를 HDFS와 하이브로 불러오는 기본적인 방법
- 스파크로 데이터를 하이브 테이블이나 스파크 작업으로 불러오는 방법
- 관계형 데이터와 HDFS 간의 데이터 송수신을 수행하는 아파치 스쿱
- 스트리밍 데이터(예를 들면 웹 로그)를 HDFS로 수집, 정송하는 아파치 플럼
- 하둡 입수 작업의 워크플로우를 관리하는 아파치 우지
- 하둡 클러스터의 데이터 관제 프레임워크인 아파치 팔콘 프로젝트

하둡은 처리할 데이터 종류가 무엇이든 HDFS에서 가져오거나, HDFS로 내보낼 수 있는 데이터 전송 도구를 활용할 수 있다. 일단 데이터가 HDFS에 저장되면 하둡 생태계의 다양한 처리 엔진으로 이 데이터를 분석할 수 있다.<br>


이 장에서는 하둡 데이터 레이크의 개념을 소개하고, 하둡으로 데이터를 입수하는 주요 도구(스파크, 스쿱, 플럼)의  개괄적인 설명과 몇 가지 구체적인 활용 예를 제시한다. 또한 데이터 입수 과정을 효과적으로 관리하는 워크플로우 도구(우지,팔콘 등)를 소개한다.

## 4.1 하둡 데이터 레이크
데이터는 어디에나 있지만, 그렇다고 데이터를 저장하고 접근하는 과정이 항상 쉬운 건 아니다. 사실, 하둡 이전에 사용한 데이터 아키텍처들은 대부분 경직된 성실이 이었고 다루거나 병경하기도 어려웠다. 하지만 데이터 레이크라는 개념은 이 모든 것을 바꾸어 놓았따.<br>
그럼 데이터 레이크란 무엇일까?<br>
기존 데이터베이스나 데이터 웨어하우스는 사전에 정의된 스키마로 데이터를 변환하는 작업이 데이터를 적재하기 전에 필요했다. 이러한 데이터 입수 단계를 '추출, 변환, 적재'라고 부르며, 이 ETL 개발 단계를 거쳐 최종 애플리케이션에서 데이터를 활용하기까지 많은 시간과 노력, 비용이 소모되는 경우가 잦았다. 무엇보다 ETL의 개발 단계에서 데이터 사용 방법을 미리 결정해야 했으며 ETL 개발이 끝난 후 이 결정을 번복하려면 많은 노력과 비용이 소모되었다. 또한 데이터가 스키마에 맞지 않거나 필요 없어 보이거나 최종 애플리케이션에 중요하지 않다는 이유 등으로 입수한 데이터를 ETL 단계에서 폐기하는 경우도 많았다.<br> 
하둡의 주요 특징 중 하나는 모든 데이터를 HDFS에 저장할 수 있는 중앙 스토리지이며, 기존 시스템보다 훨씬 낮은 비용으로 대규모 데이터셋을 중복 저장할 수 있다는 것이다.<br>
즉, 하둡 데이터 레이크란 조직의 모든 데이터를 원시 형식으로 저장하는 중앙 스트리지를 말한다. 데이터 레이크는 하둡 애플리케이션이 데이터를 처리하는 시점에 이르러서야 비로소 종전의  ETL 단계와 유사한 데이터 변환 과정이 수행된다. 이러한 방식을 '스키마 온 리드'라고 하며, 프로그래머와 사용자가 원시 데이터에 자신이 원하는 구조를 적용할 수 있다는 장점이 있다. 반면, 기존 데이터 웨어하우스는'스키마 온 라이트'방식을 사용하며, 데이터가 어떻게 활용될지 사전에 추측하는 설계 단계를 반드시 거쳐야 한다.<br>
데이터 과학에서는 프로젝트에 중요한 데이터가 무엇인지 사전에 명확하게 파악하기 어려우므로 모든 데이터를 원시 형태로 유지하면 많은 이점을 누릴 수 있다.<br>
기존의 데이터 웨어하우스에 비해 데이터 레이크는 빅데이터 관점에서 다음 세 가지 이점이 있다.
- 모든 데이터를 사용할 수 있다. 향후 데이터를 어떻게 사용할지를 사전에 추측할 필요가 없다.
- 모든 데이터를 공유할 수 있다. 모든 데이터가 모든 실무자와 연구자에게 열려 있다. 이전에는 이기종 시스템의 데이터 구획화 때문에 일부 데이터에 쉽게 접근할 수 없었다.
- 모든 데이터 접근 방식이 가능하다. 다양한 처리 엔진(맵리듀스, 테즈, 스파크)과 애플리케이션(하이브,스파크 SQL, 피그)을 이용해 데이터를 탐색하고 필요에 따라 처리할 수 있다.

하지만 데이터 웨어하우스는 분명 유용한 비즈니스 도구이며, 하둡은 이를 대체한다기보다는 보완하도록 설계되었다. 그럼에도 기존 데이터 웨어하우스는 새로운 유형의 데이터가 등장한 빅데이터 시대 이전에 개발된 기술이다. 이미 소셜 미디어 데이터, 클릭 로그, 센서 데이터 등 여러 유형의 데이터가 데이터 레이크를 빠르게 채우기 시작했다. 기존 ETL 단계로는 이 같은 유형의 데이터가 유입되는 속도를 따라잡지 못할 것이다. 따라서 데이터 웨어하우스와 데이터 레이크 기술은 각각의 설계 목적에 맞는 요구 사항을 만족시키며 앞으로도 계속 공존할 것이다.<br>
그림에서는 세 데이터 소스(A, B, C)에서 생성되는 데이터가 ETL 과정과 데이터 레이크에 각각 유입된다. ETL 과정은 데이터를 관계형 데이터베이스에 저장하면서 스키마에 맞게 변환한다.(스키마 온 라이드). 반면, 데이터 레이크는 원시 형태 그대로 저장하며, 하둡 애플리케이션이 데이터 레이크에서 데이터를 읽어 들이는 시점에 이르러서야 비로소 스키마를 적용한다. ETL 단계에서는 몇몇 데이터를 버리는 경우가 있다. 물론 데이터 웨어하우스의 사용자도 원하는 데이터에 접근할 수 있다. 하지만 하둡에서는 데이터 레이크에 데이터가 입수되는 즉시 데이터에 접근 할 수 있다.

## 4.2 HDFS
모든 하둡 애플리케이션은 하둡 분산 파일 시스템에 저장된 데이터를 바탕으로 실행된다. HDFS의 파일 작업은 대부분의 사용자에게 익숙한 로컬 파일 시스템과 분리돼 있다. 즉, 사용자는 필요한 파일을 로컬 파일 시스템에서 HDFS로 직접 복사하거나, HDFS에서 로컬 파일 시스템으로 복사해야 한다. HDFS는 일반적인 파일 시스템이 아니며 기존의 POSIX 또는 이와 유사한 파일 시스템을 대체할 수 없다.<br>
HDFS는 대용량 파일의 읽기 및 쓰기 작업에 최적화된 독특한 스트리밍 파일 시스템이다. HDFS에 파일을 저장하면, HDFS는 파일의 데이터를 슬라이스로 분할해 하둡 클러스터의 여러 서버에 이중으로 저장한다. 데이터의 분할 과정에서는 대용량 파일을 작은 단위의 블록으로 나눠 클러스터 노드에 '투명하게'저장한다. HDFS는 여러 슬라이스를 동시에 병렬로 처리하므로 파일 연산을 더 빠르게 처리할 수 있다. 사용자는 실제 파일 슬라이스를 직접 볼 수없지만, 모든 HDFS파일을 마치 일반 파일 시스템의 파일처럼 다룰 수 있다(파일 복사, 이동, 삭제 등). HDFS에서 호스트 파일 시스템으로 파일을 전송할 때는 분할된 슬라이스를 다시 합쳐 파일 하나로 만든 후 호스트에 저장한다. <br>
파일 슬라이스가 여러 서버에 이중으로 저장되므로 일부 서버 장애가 데이터 유실로 이어지지 않는다. HDFS는 랜덤 읽기나 쓰기를 지원하지 않는 대신 이어 쓰기를 지원하게 설계되었다. 또한 테스트 목적으로 랩톱이나 데스크톱 컴퓨터의 하드 드라이브에도 HDFS를 설치할 수 있다. 하지만 이 경우 파일 분할이나 이중화가 적용되지 않든다.

## 4.3 파일을 HDFS로 직접 정상하기
HDFS와 데이터를 주고받는 가장 쉬운 방법은 기본 HDFS 명령을 사용하는 것이다. HDFS 명령은 HDFS 파일 시스템을 연동한 일종의 래퍼이며, 로컬 명령(cp. ls. mv등)은 오직 로컬 파일 시스템에서만 사용할 수 있다. 다음과 같이 HDFS의 put 명령으로 로컬 파일 시스템에 저장된 test 파일을 HDFS로 복사할 수 있다. <br>
- $ hdfs dfs -put test <br>


다음과 같이 HDFS에 저장된 파일 목록을 조회할 수 있다. HDFS 파일 목록은 로컬 파일 시스템에서 ls -l 명령이 출력하는 결과와 유사하다. <br>
- $ hdfs dfs -ls <br>
-rw-r--r--  2 username hdfs    497  2016-05-11 14:32 test <br>

HDFS에서 로컬 파일 시스템 파일(another-test)을 복사할 때는 다음과 같이 get 명령을 사용한다.
- $ hdfs dfs -get another-test

다른 HDFS 명령도 예제에서 살펴볼 것이다. 부록 A.1 HDFS 퀙스타트에서 기본 HDFS 명령 예제(파일 조회, 복사 삭제 등)를 볼 수 있다.

## 4.4 파일을 하이브 테이블로 가져오기
아파치 하이브 SQL과 유사한 스크립트를 사용해 HDFS의 데이터를 분석하는 도구다. 데이터과학 프로젝트에서 프프렛드시트나 데이터베이스에서 추출한 텍스트 파일을 하이브로 가져 와야 하는 경우가 종종 있다. 텍스트 파일은 TSV, CSV, JSON형식, 또는 단순한 비표준 텍스트 파일일 수도 있다. 일단 데이터를 하이브 테이블로 가져오면, 나중에 진행할 모델링 단계에서 이를 편리하게 사용할 수 있다. 5장 하둡을 활용한 데이터 개조에서 더 자세히 다룰 특징 변수 생성을 대표적인 모델링 단계로 볼 수 있다.<br>
일단 하이브 테이블로 데이터를 가져오면 하이브의 SQL 쿼리나 피그, 스파크 같은 다양한 도구를 사용해 데이터를 처리할 수 있다.<br>  
하이브는 두 종류의 테이블을 지원한다. 첫 번째는 내부 테이블이며, 하이브가 완전히 관리하는 테이블이다. 내부 테이블을 삭제하면 하이브의 테이블 메타 정의와 테이블에 들어 있는 모든 데이터도 같이 삭제되다. 내부 테이블은 ORC 같은 최적화된 형식으로 저장될 수 있어 비교적 빠른 성능을 보인다. 두 번째는 외부 테이블이며, 하이브가 직접 관리하지 않는다. 외부 테이블은 오직 하이브의 메타 정의만 사용해 원시 형태로 저장된 텍스트 데이터에 접근한다. 내부 테이블과는 달리, 외부 테이블의 데이터를 삭제하면 하이브의 테이블 메타 정의만 섹제되고 실제 데이터는 그대로 유지된다. 외부 테이블은 해당 데이터가 하이브 외부에 적재(예를 들어 다른 애플리케이션도 파일을 사용, 생성, 관리하는 경우)돼 있거나, 테이블이 삭제되더라도 원본 데이터가 본래 위치에 남아야 할 때 주로 사용한다.<br>
하이브를 활용하는 사례가 많은 관계로 여기서는 하이브가 지원하는 모든 입수 방법을 다루기 보다는 CSV 파일을 하이브로 가져오는 기본적인 예제를 소개한다. 더 많은 내용은 하이브 프로젝트 페이지(https://hive.apache.org)를 참조하기 바란다.

### 4.4.1 CSV 파일을 하이브 테이블로 가져오기